## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [5]:
data_dir = "../../Data/Weather"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [6]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3
0,-0.024993,-0.124485,0.106647,0.152136
1,-0.257769,0.293571,-0.032213,-0.000325
2,0.018043,-0.061463,0.051560,-0.012518
3,0.004450,-0.034110,0.122845,0.092054
4,-0.041670,-0.005613,-0.117826,0.219750


In [7]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00087395', u'USW00012833', u'USC00082008', u'USW00012873', u'USW00012816']


,0,1,2,3
station,,,,
USC00087395,-0.024993,-0.124485,0.106647,0.152136
USW00012833,-0.257769,0.293571,-0.032213,-0.000325
USC00082008,0.018043,-0.061463,0.051560,-0.012518
USW00012873,0.004450,-0.034110,0.122845,0.092054
USW00012816,-0.041670,-0.005613,-0.117826,0.219750


In [8]:
table=Eig.join(Stations,how='left')

In [12]:
#table=table[['name','latitude','longitude','elevation',0,1,2,3,4,5,6,7]]
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00087395,PUNTA GORDA,26.9333,-82.0500,3.0,-0.024993,-0.124485,0.106647,0.152136
USW00012833,CROSS CITY AP,29.6333,-83.1053,11.6,-0.257769,0.293571,-0.032213,-0.000325
USC00082008,CROSS CITY 1 E,29.6333,-83.1053,12.8,0.018043,-0.061463,0.051560,-0.012518
USW00012873,ST PETERSBURG INTL AP,27.9106,-82.6875,3.4,0.004450,-0.034110,0.122845,0.092054


In [13]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [23]:
#find the min lat, max lat..
print table.min()
print table.max()

name          BAYPORT
latitude      24.6278
longitude    -83.3883
elevation      -999.9
0           -0.306944
1           -0.361915
2           -0.441541
3           -0.360346
dtype: object
name         WEEKI WACHEE
latitude          30.1064
longitude        -82.0333
elevation            73.2
0               0.0455826
1                0.293571
2                 0.16194
3                0.271551
dtype: object


In [24]:
min_lat,max_lat,min_long,max_long = box = (24.6278, 30.1064, -83.3883, -82.0333)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [25]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined